In [1]:
import re
from pathlib import Path

In [2]:
env_storage_path = Path(r'C:\temp\envs')

In [9]:
if not env_storage_path.exists():
    env_storage_path.mkdir()

- Get list of current Anaconda environments

In [3]:
env_str = !conda env list

- Remove directory path from environment name.

**Note:** This does not work for environments with spaces in the name.

In [4]:
env_names = [name.split(' ',1)[0] for name in env_str]

- Remove list items that do not start with a letter
(This gets rid of comments and blank names.)

In [5]:
starts_with_letter = re.compile(r'^[a-z]', flags=re.IGNORECASE)
env_names = [name for name in env_names if starts_with_letter.match(name) ]

- Store environment info for each environment

In [10]:
for this_env in env_names:
    print(f'Storing environment for {this_env}')
    spec_file = env_storage_path / f'{this_env}_specfile.txt'
    full_yml_file = env_storage_path / f'{this_env}_FULL.yml'
    history_yml_file = env_storage_path / f'{this_env}.yml'
    
    !conda activate "{this_env}"

    !conda list --explicit > "{str(spec_file)}"
    !conda env export > "{str(full_yml_file)}"
    !conda env export --from-history > "{str(history_yml_file)}"


Storing environment for base
Storing environment for Graphs
Storing environment for Standard
Storing environment for C2
Storing environment for EDW_QA
Storing environment for FormExtraction
Storing environment for McMedHack
Storing environment for TestPathDev
Storing environment for sectionary
Storing environment for sectionaryDev
Storing environment for variandb_relations


- Manual mode for environments with spaces

In [12]:
env_names

['base',
 'Graphs',
 'Standard',
 'C2',
 'EDW_QA',
 'FormExtraction',
 'McMedHack',
 'TestPathDev',
 'sectionary',
 'sectionaryDev',
 'variandb_relations']

In [17]:
this_env = 'Form Extraction'
print(f'Storing environment for {this_env}')
spec_file = env_storage_path / f'{this_env}_specfile.txt'
full_yml_file = env_storage_path / f'{this_env}_FULL.yml'
history_yml_file = env_storage_path / f'{this_env}.yml'

!conda activate "{this_env}"

!conda list --explicit > "{str(spec_file)}"
!conda env export > "{str(full_yml_file)}"
!conda env export --from-history > "{str(history_yml_file)}"

Storing environment for Form Extraction


## Creation command

In [ ]:
install_envs = [
    'Graphs',
     'Standard',
     'TestPathDev',
     'sectionary',
     'sectionaryDev',
     'variandb_relations'
    ]

In [ ]:
for make_env in install_envs:
    print(f'Creating environment for {make_env}')
    spec_file = env_storage_path / f'{make_env}_specfile.txt'
    !conda create --name "{make_env}" --file "{spec_file}"
    !conda activate "{make_env}"
    !xlwings license update -k noncommercial
    !xlwings addin install
    !xlwings config create --force


In [ ]:
#spec_file = env_storage_path / f'{this_env}_specfile.txt'
#!conda create --name "{this_env}" --file "{spec_file}"


## xlwings Configure commands

In [ ]:
#!conda activate "{this_env}"
#!xlwings license update -k noncommercial
#!xlwings addin install
#!xlwings config create --force


## Removing environment commands

In [ ]:
#!conda activate {this_env}
#!xlwings addin uninstall

#!conda deactivate
#!conda remove --name {this_env} --all
